In [1]:
# Импортирование библиотек для иницализации основных параметров системы
from fortran_calc_tools import *

# Импортирование неодходимых библиотек
from os import path 
import time
import pandas as pd, numpy as np
from collections import namedtuple
from calc_tools import pairwise_1, pairwise_2
from scipy.linalg import logm, expm, sqrtm, eig, lu, lu_factor, lu_solve
from multiprocessing import Pool
from numpy.linalg import inv
from numpy import linalg as LA

# Импортирование библиотек для визуализации данных !!!!!
import matplotlib.pyplot as plot
import matplotlib as mpl
import seaborn as sns
import plotly as pt
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go

In [2]:
# Задание значении характеристик материала для фаз и троса
#                         GM    OPM    S     XA   YA
Faza_A = Fazy("Фаза А", 35.336, 1.0, 150.0, 0.0, 19.0)
Faza_B = Fazy("Фаза В", 35.336, 1.0, 150.0, 6.3, 19.0)
Faza_C = Fazy("Фаза С", 35.336, 1.0, 150.0, 4.2, 25.0)
Tross = Fazy("Тросс", 17.336, 4000.0, 50.0, 2.1, 28.0)

# В данном случае у нас будет 50 гармоник от [1 до 50]
garmoniki = Garmoniki(50)

# Создание объекта для проведения расчетов
rashet = Rachety()

# Инициализация объекта для подстанции "Зеленая" и чтение файла
basepath = path.abspath('')
filepath = path.abspath(path.join(basepath, "..", path.join("data", "Данные ПС Сарепта1.xls")))
PodS_zelenaya = PodStans(filepath, volt=110, harm_num=49)

dummy_str_input = "fuzzBuzz for testing"
dummy_int_input = 404

system = System(PodS_zelenaya, garmoniki, rashet, Fazy, dummy_str_input, dummy_str_input, dummy_int_input, dummy_int_input,
     dummy_int_input, dummy_int_input, dummy_int_input)

# Установка конфигурации для выведения данных
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [8]:
# GETTING MAIN LINE CHARACTERISTICS FROM EXCEL FILE
UM, FUM, AIM, FIM, funu, fu, funi, fi, knsu, knsi, rmsu, rmsi = system.line_matrix(1)

In [9]:
# DECLARING VARIABLES

WD = np.zeros((2, 50))
PD = np.zeros((3, 700))

UM1 = np.zeros((700, UM.shape[1],UM.shape[2]))
UM2 = np.zeros((700, UM.shape[1],UM.shape[2]))
AIM1 = np.zeros((700, UM.shape[1],UM.shape[2]))
AIM2 = np.zeros((700, UM.shape[1],UM.shape[2]))
PPR1, PPR2 = np.zeros((700)), np.zeros((700))

PPP = np.zeros((1000, 50)); PPP1 = np.zeros((1000, 50));
PPP2 = np.zeros((1000, 50)); PPP3 = np.zeros((1000, 50));
PPP4 = np.zeros((1000, 50)); PPP5 = np.zeros((1000, 50));
PPP6 = np.zeros((1000, 50)); PPP7 = np.zeros((1000, 50));
PPP8 = np.zeros((1000, 50));

# !!!!!!!!!!!!!!!!! CordN = 8 was in original here 
cordN = 8
UK1 = np.array([complex(0, 0) for _ in range(cordN)], dtype="complex128")
AIK1 = np.array([complex(0, 0) for _ in range(cordN)], dtype="complex128")

UK10 = complex(0, 0); UK11 = complex(0, 0); UK12 = complex(0, 0);
AIK10 = complex(0, 0); AIK11 = complex(0, 0); AIK12 = complex(0, 0);
AL = complex(-0.5, 0.866025);

PP1 = 0.0
PP2 = 0.0
RZ = 35.3

In [10]:
# Implementing RASCHET fortran's subroutine in python
def RASCHET(NN, LL):
# LL is in range [0-559] NN [0-49]

#     print(NN, LL)
    global UK1, AIK1, PPP,PP1,PP2,PPP1,PPP2,PPP3,PPP4,PPP5,PPP6,PPP7,PPP8, MM,M,M1,MT,M10,M20,PR,K1,K2,K3,N1,N2,N3,MPR,MTR,MMT
    
    
    XA, YA, OMP, GM, S = Fazy.construc_matrices()
    R0, R, UXM, HI, R11, DET2, DET4 = np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M)), np.zeros((M))
    EVU, B, UX, AIX, SM, EVI, DET1, DET3, BB, AIXM, AA = np.zeros((M)), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M)), np.zeros((M), dtype="complex128"), np.zeros((M), dtype="complex128"), np.zeros((M)), np.zeros((M)), np.zeros((M)),
    B1, B4, B5 = np.zeros((M20), dtype="complex128"), np.zeros((M20)), np.zeros((M20), dtype="complex128")
    B6, B10, B7 = np.zeros((M10)), np.zeros((M10), dtype="complex128"), np.zeros((M10)) 
    
    DET10, DET20, SS, SS1, EX1,  = [], [], [], [], []
    
    HC1, HC2, HC3, HC4, F10, XL, XL1, G, D, HC, UXM = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)) 
    Z, Y, AU, E,  AAI, F, F1 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    F2, D1, D2, D3, LU, LI = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    LU1, LI1, LU2, LU3, LI2, LI3 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    F3, F4, F5, F6, F7 = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M))
    HH13, HH14, HH21, HH22, HH23 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    HH31, HH32, HH33, HH34, HH24 = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    HH41, HH42, HH43, HH44, HH = np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128"), np.zeros((M10, M10))
    CC, DD, HH11, HH12 = np.zeros((M, M)), np.zeros((M, M)), np.zeros((M, M), dtype="complex128"), np.zeros((M, M), dtype="complex128")
    A1, A2 = np.zeros((M1, M1), dtype="complex128"), np.zeros((M1, M1), dtype="complex128")
    
    AG, GG = np.zeros((M1, M, M), dtype=type), np.zeros((M20, M20), dtype="complex128")
    GG1, GG2, GG3, GG4, GG5 = np.zeros((M20, M20), dtype="complex128"), np.zeros((M20, M20), dtype="complex128"), np.zeros((M10, M20), dtype="complex128"), np.zeros((M10, M10), dtype="complex128"), np.zeros((M10, M10), dtype="complex128")
    IPVT1, IH = np.zeros((M1)), np.array([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0])
    
    if PR == 1:
        PP1 = 0
    if PR == 2:
        PP2 = 0
#     PPP[NN, LL] = 0
#     PPP1[NN, LL] = 0
#     PPP2[NN, LL] = 0
#     PPP3[NN, LL] = 0
#     PPP4[NN, LL] = 0
#     PPP5[NN, LL] = 0
#     PPP6[NN, LL] = 0
#     PPP7[NN, LL] = 0
#     PPP8[NN, LL] = 0
    
    MMT = MM / MT
    W = float(LL+1)
    EX1 = complex(2.71828, 0)
    
    # WRITING INTO UNIT 5. LINE MAIN CHARACTERISTIC
    
    for i in range(M):
        R[i]=np.sqrt(S[i]/np.pi)/1000.
        HI[i]=R[i]/(2.)*np.sqrt(2*np.pi*W*50*4*np.pi*OMP[i]*GM[i]/20)
        R0[i]=1000/(GM[i]*S[i])
        if HI[i] < 1:
            R11[i]=R0[i]*(1+HI[i]**4/3)
        if HI[i] > 1: 
            R11[i]=R0[i]*(HI[i]+0.25+3./(64.*HI[i]))
        if i+1 == M:
            pass
            # WRITE R11's values INTO THE FILE "R11"

    
    # 845 - 12
    for i in range(M10):
        for j in range(M10):
            HH[i,j] = 0 

    # 12 - 161
    for i in range(M):
        for j in range(M):
            if i == j:
                D[i,i] = R[i]
            if i != j:
                D[i,j]=np.sqrt((XA[i]-XA[j])**2+(YA[i]-YA[j])**2)
            HC[i,j]=np.sqrt((XA[i]-XA[j])**2+(YA[i]+YA[j])**2)
            E[i,j]=complex(0.0, 0.0)
            E[i,i]=complex(1.0, 0.0)

    # 161 - 740
    for i in range(M):
        for j in range(M):
            XL1[i,j]=(0.145*np.log10(1000/D[i,j]))/(100*np.pi)
    
    # 740 - 743
    for i in range(M):
        for j in range(M):
            HC1[i,j]=41.4*10**6*np.log10(HC[i,j]/D[i,j])
    
    # 743 - 744
    HC1 = HC1.astype('float64')
    HC3 = DLINRG(HC1)
    F10 = DMRRRR(HC1,HC3)
    
    for i in range(M):
        for j in range(M):
            HC2[i,j]=HC3[i,j]*2.*np.pi*50
    
    # 744 - 847
    for i in range(M):
        for j in range(M):
            XL[i,j]=XL1[i,j]*W*2*50*np.pi
            HC4[i,j]=HC2[i,j]*W
            R10=0.0
            if i == j:
                Z[i,j]=complex(R11[i],XL[i,j]) 
            if i != j:
                Z[i,j]=complex(R10,XL[i,j])
            if i == j:
                G[i,j]=0.00000004*YA[i]/YA[i] 
            if i != j:
                G[i,j]=-0.00000004*YA[0]/D[i,j] 
            G[i,j]=0.
            Y[i,j]=complex(G[i,j],HC4[i,j])
    
    debug("Y", Y)
    # 847 - the end of subroutine        
    for III in range(MT): # <----- main loop # 1300
        
        if M != 3: # GOTO 767            
            if (M != 4): # GOTO 768                
                if (M != 6): # GOTO 769
                    if (M != 7): # GOTO 770
                        if (M != 8): # GOTO 771
                            pass

                        else:
                            # 770 - 766
                            for i in range(3):
                                B5[i]=UK1[i]
                                B5[i+3]=UK1[i]
                                B5[MPR+1-1]=complex(0.,0.)
                                B5[MPR+2-1]=complex(0.,0.)
                                B5[i+M]=AIK1[i]
                                B5[i+M+3]=AIK1[i]
                                B5[2*M-1-1]=complex(0.,0.)
                                B5[2*M-1]=complex(0.,0.)
                                B5[i+2*M]=complex(0.,0.)
                                B5[i+2*M+3]=complex(0.,0.)
                                B5[3*M-1-1]=complex(0.,0.)
                                B5[3*M-1]=complex(0.,0.)
                                B5[i+3*M]=complex(0.,0.)
                                B5[i+3*M+3]=complex(0.,0.)
                                B5[4*M-1-1]=complex(0.,0.)
                                B5[4*M-1]=complex(0.,0.)
                          
                    else:
                        # 769 - 765
                        for i in range(3):
                            B5[i]=UK1[i]
                            B5[i+3]=UK1[i] 
                            B5[M-1]=complex(0.,0.) 
                            B5[i+M]=AIK1[i]
                            B5[i+M+3]=AIK1[i] 
                            B5[2*M-1]=complex(0.,0.)
                            B5[i+2*M]=complex(0.,0.)
                            B5[i+2*M+3]=complex(0.,0.)
                            B5[3*M-1]=complex(0.,0.)
                            B5[i+3*M]=complex(0.,0.)
                            B5[i+3*M+3]=complex(0.,0.)
                            B5[4*M-1]=complex(0.,0.)
                      

                else:
                    # 768 - 764
                    for i in range(3):
                        B5[i]=UK1[i]
                        B5[i+3]=UK1[i]
                        B5[i+M]=AIK1[i]
                        B5[i+M+3]=AIK1[i]
                        B5[i+2*M]=complex(0.,0.)
                        B5[i+2*M+3]=complex(0.,0.)
                        B5[i+3*M]=complex(0.,0.)
                        B5[i+3*M+3]=complex(0.,0.)

            else:
#               CONTSTATNLY RUNS THIS BRANCH
                # 767 - 762
                for i in range(3):
                    B5[i]=UK1[i]
                    B5[M-1]=complex(0.,0.)
                    B5[i+M]=AIK1[i]
                    B5[2*M-1]=complex(0.,0.)
                    B5[i+2*M]=complex(0.,0.)
                    B5[3*M-1]=complex(0.,0.)
                    B5[i+3*M]=complex(0.,0.)
                    B5[4*M-1]=complex(0.,0.)
                
        else:
            # 847 - 761
            for i in range(3):
                B5[i]=UK1[i]
                B5[i+3]=AIK1[i]
                B5[i+6]=complex(0.,0.)
                B5[i+9]=complex(0.,0.)
            
# &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&& DEBUGGING &&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&
        
        AU = DMCRCR(Z,Y)
        SS1=np.sqrt(AU[0,0])
        EVU = DEVLCG(AU)
        
        # IMPORTANT POINT
        for j in range(M):
            for i in range(M):
                F[i,j]=EVU[i]**j # (j-1) was in the first case
                F1[i,j]=F[i,j]
        
        for i in range(M1):
            for j in range(M1):
                A1[i,j]=F[i,j]

        A2 = DLFTCG(A1)
        SS = DET10 = DLFDCG(A2); DET20 = 1 # parameters DET10,DET20 were in function call
        # SS=DET10*(10.**DET20)
        
        for j in range(M1):

            for ii in range(M1):
                for jj in range(M1):
                    F[ii,jj]=F1[ii,jj]

            for i in range(M1):
                F[i,j]=EVU[i]**0.5

            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET1[j] = DLFDCG(A2); DET2[j] = 1 # parameters DET1(J), DET2(J) were in function call

        F2 = DMCRCR(AU,AU)
        F3 = DMCRCR(F2,AU)
        F4 = DMCRCR(F3,AU)
        F5 = DMCRCR(F4,AU)
        F6 = DMCRCR(F5,AU)
        F7 = DMCRCR(F6,AU)
        
        for i in range(M1):
            for ii in range(M):
                for jj in range(M):

                    if i == 0:
                        AG[0,ii,jj]=E[ii,jj]
                    if i == 1:
                        AG[1,ii,jj]=AU[ii,jj]
                    if i == 2:
                        AG[2,ii,jj]=F2[ii,jj]
                    if i == 3:
                        AG[3,ii,jj]=F3[ii,jj]
                    if i == 4:
                        AG[4,ii,jj]=F4[ii,jj]
                    if i == 5:
                        AG[5,ii,jj]=F5[ii,jj]
                    if i == 6:
                        AG[6,ii,jj]=F6[ii,jj]
                    if i == 7:
                        AG[7,ii,jj]=F7[ii,jj]

        for ii in range(M):
            for jj in range(M):
                LU[ii,jj]=0.

        for i in range(M1):
            DET3[i]=DET1[i]/DET10
            DET4[i]=DET2[i]-DET20
            for ii in range(M):
                for jj in range(M):
                    LU[ii,jj]=LU[ii,jj]+AG[i,ii,jj]*DET3[i]*(10**DET4[i])
        
        F3 = DMCRCR(LU,LU)
        AAI = DMCRCR(Y,Z)
        EVI = DEVLCG(AAI)
  
        for j in range(M):
            for i in range(M):
                F[i,j] = EVI[i]**j # j-1
                F1[i,j] = F[i,j]

        for i in range(M1):
            for j in range(M1):
                A1[i,j]=F[i,j]

        A2 = DLFTCG(A1)
        DET10 = DLFDCG(A2); DET20 = 1 # parameters DET10,DET20 were in function call
        
        for j in range(M1):
            for ii in range(M1):
                for jj in range(M1):
                    F[ii,jj]=F1[ii,jj]

            for i in range(M1):
                F[i,j]=EVI[i]**0.5
            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET1[j] = DLFDCG(A2); DET2[j] = 1 # parameters DET1(J),DET2(J) were in function call
            
        F2 = DMCRCR(AAI,AAI)
        F3 = DMCRCR(F2,AAI,)
        F4 = DMCRCR(F3,AAI)
        F5 = DMCRCR(F4,AAI)
        F6 = DMCRCR(F5,AAI)
        F7 = DMCRCR(F6,AAI)

        for i in range(M1):
            for ii in range(M):
                for jj in range(M):

                    if i == 0:
                        AG[0,ii,jj]=E[ii,jj]
                    if i == 1:
                        AG[1,ii,jj]=AAI[ii,jj]
                    if i == 2:
                        AG[2,ii,jj]=F2[ii,jj]
                    if i == 3:
                        AG[3,ii,jj]=F3[ii,jj]
                    if i == 4:
                        AG[4,ii,jj]=F4[ii,jj]
                    if i == 5:
                        AG[5,ii,jj]=F5[ii,jj]
                    if i == 6:
                        AG[6,ii,jj]=F6[ii,jj]
                    if i == 7:
                        AG[7,ii,jj]=F7[ii,jj]

        for ii in range(M):
            for jj in range(M):
                LI[ii,jj]=0.

        for i in range(M1):
            for ii in range(M):
                for jj in range(M):
                    LI[ii,jj]=LI[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))
        
        F3 = DMCRCR(LI,LI)
        LM=MMT
        
        for n, _ in enumerate(range(2), start=1):

            if n == 1:
                SA=-1.
            if n == 2:
                SA=1.

            for ii in range(M):
                for jj in range(M):
                    LU1[ii,jj]=SA*MMT*LU[ii,jj]


            EVU = DEVLCG(LU1)
            for i in range(M):
                for j in range(M):
                    F[i,j]=EVU[i]**j # (j-1) here was an expression (j-1)


            for ii in range(M):
                for jj in range(M):
                    F1[ii,jj]=F[ii,jj]


            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]
            
            A2 = DLFTCG(A1)
            DET10 = DLFDCG(A2); DET20 = 1 # DET10 and DET20 were the arguments of this function
            
            for j in range(M):
                for ii in range(M):
                    for jj in range(M):
                        F[ii,jj]=F1[ii,jj]


                for i in range(M):
                    F[i,j]=EX1**EVU[i]


                for ii in range(M1):
                    for jj in range(M1):
                        A1[ii,jj]=F[ii,jj]


                A2 = DLFTCG(A1)
                DET1[j] = DLFDCG(A2); DET2[j] = 1
            
            F2 = DMCRCR(LU1,LU1)
            F3 = DMCRCR(F2,LU1)
            F4 = DMCRCR(F3,LU1)
            F5 = DMCRCR(F4,LU1)
            F6 = DMCRCR(F5,LU1)
            F7 = DMCRCR(F6,LU1)

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):

                        if i == 0:
                            AG[0,ii,jj]=E[ii,jj]
                        if i == 1:
                            AG[1,ii,jj]=LU1[ii,jj]
                        if i == 2:
                            AG[2,ii,jj]=F2[ii,jj]
                        if i == 3:
                            AG[3,ii,jj]=F3[ii,jj]
                        if i == 4:
                            AG[4,ii,jj]=F4[ii,jj]
                        if i == 5:
                            AG[5,ii,jj]=F5[ii,jj]
                        if i == 6:
                            AG[6,ii,jj]=F6[ii,jj]
                        if i == 7:
                            AG[7,ii,jj]=F7[ii,jj]

            for ii in range(M):
                for jj in range(M):
                    LU2[ii,jj]=0.

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        LU2[ii,jj]=LU2[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))

            if n == 2:
                F2 = DMCRCR(LU3,LU2)
                continue
            else:
                for ii in range(M):
                    for jj in range(M):
                        LU3[ii,jj]=LU2[ii,jj]

        for n in range(2):
            if n == 0:
                SA=-1
            if n == 1:
                SA=1

            for ii in range(M):
                for jj in range(M):
                    LI1[ii,jj]=SA*MMT*LI[ii,jj]

            EVI = DEVLCG(LI1)

            for i in range(M):
                for j in range(M):
                    F[i,j]=EVI[i]**j # J-1 in original

            for ii in range(M):
                for jj in range(M):
                    F1[ii,jj]=F[ii,jj]

            for ii in range(M1):
                for jj in range(M1):
                    A1[ii,jj]=F[ii,jj]

            A2 = DLFTCG(A1)
            DET10 = DLFDCG(A2); DET20 = 1
            
            for j in range(M):
                for ii in range(M):
                    for jj in range(M):
                        F[ii,jj]=F1[ii,jj]

                for i in range(M):
                    F[i,j]=EX1**EVI[i]

                for ii in range(M1):
                    for jj in range(M1):
                        A1[ii,jj]=F[ii,jj]

                A2 = DLFTCG(A1)
                DET1[j] = DLFDCG(A2); DET2[j] = 1


            F2 = DMCRCR(LI1,LI1)
            F3 = DMCRCR(F2,LI1)
            F4 = DMCRCR(F3,LI1)
            F5 = DMCRCR(F4,LI1)
            F6 = DMCRCR(F5,LI1)
            F7 = DMCRCR(F6,LI1)
            
            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        if i == 0:
                            AG[0,ii,jj]=E[ii,jj]
                        if i == 1:
                            AG[1,ii,jj]=LI1[ii,jj]
                        if i == 2:
                            AG[2,ii,jj]=F2[ii,jj]
                        if i == 3:
                            AG[3,ii,jj]=F3[ii,jj]
                        if i == 4:
                            AG[4,ii,jj]=F4[ii,jj]
                        if i == 5:
                            AG[5,ii,jj]=F5[ii,jj]
                        if i == 6:
                            AG[6,ii,jj]=F6[ii,jj]
                        if i == 7:
                            AG[7,ii,jj]=F7[ii,jj]

            for ii in range(M):
                for jj in range(M):
                    LI2[ii,jj]=0

            for i in range(M1):
                for ii in range(M):
                    for jj in range(M):
                        LI2[ii,jj]=LI2[ii,jj]+(DET1[i]*(10**DET2[i])*AG[i,ii,jj])/(DET10*(10**DET20))
            
            if n == 2:
                continue
            else:
                for ii in range(M):
                    for jj in range(M):
                        LI3[ii,jj]=LI2[ii,jj]


        F2 = DMCRCR(LI3,LI2)
        
        for i in range(M20):
            for j in range(M20):
                GG[i,j]=0.
                GG1[i,j]=0.
                GG2[i,j]=0.

        for i in range(M):
            GG[i,i]=1.
            GG[i,i+M]=1.
            GG[i+M,i+2*M]=1.
            GG[i+M,i+3*M]=1.
            GG1[i,i]=1.
            GG1[i,i+M]=1.
            GG1[i+M,i+2*M]=1.
            GG1[i+M,i+3*M]=1.
        
        for i in range(M):
            for j in range(M):
                GG[i+2*M,j]=-LU[i,j]
                GG[i+2*M,j+M]=LU[i,j]
                GG[i+3*M,j+2*M]=-1*LI[i,j]
                GG[i+3*M,j+3*M]=+LI[i,j]

                GG1[i+2*M,j]=LU3[i,j]
                GG1[i+2*M,j+M]=LU2[i,j]
                GG1[i+3*M,j+2*M]=LI3[i,j]
                GG1[i+3*M,j+3*M]=LI2[i,j]

        GG2 = DLINCG(M20,GG1) # Look for M20

        for i in range(M):
            for j in range(M):
                HH11[i,j]=GG2[i,j]
                HH12[i,j]=GG2[i,j+M]
                HH13[i,j]=GG2[i,j+2*M]
                HH14[i,j]=GG2[i,j+3*M]
                HH21[i,j]=GG2[i+M,j]
                HH22[i,j]=GG2[i+M,j+M]
                HH23[i,j]=GG2[i+M,j+2*M]
                HH24[i,j]=GG2[i+M,j+3*M]
                HH31[i,j]=GG2[i+2*M,j]
                HH32[i,j]=GG2[i+2*M,j+M]
                HH33[i,j]=GG2[i+2*M,j+2*M]
                HH34[i,j]=GG2[i+2*M,j+3*M]
                HH41[i,j]=GG2[i+3*M,j]
                HH42[i,j]=GG2[i+3*M,j+M]
                HH43[i,j]=GG2[i+3*M,j+2*M]
                HH44[i,j]=GG2[i+3*M,j+3*M]
        
        F = DMCRCR(LU3,HH11)
        HH11 = DMCRCR(LU,F)
        F = DMCRCR(LU2,HH21)
        HH21 = DMCRCR(LU,F)
        F = DMCRCR(LU3,HH13)
        HH13 = DMCRCR(LU,F)
        F = DMCRCR(LU2,HH23)
        HH23 = DMCRCR(LU,F)
        F = DMCRCR(LI3,HH32)
        HH32 = DMCRCR(LI,F)
        F = DMCRCR(LI2,HH42)
        HH42 = DMCRCR(LI,F)
        F = DMCRCR(LI3,HH34)
        HH34 = DMCRCR(LI,F)
        F = DMCRCR(LI2,HH44)
        HH44 = DMCRCR(LI,F)
        
        for i in range(M10):
            for j in range(M20):
                GG3[i,j]=0.0

        for i in range(M):
            for j in range(M):
                GG3[i,j]=-1*HH11[i,j]+HH21[i,j]
                GG3[i,j+2*M]=-1*HH13[i,j]+HH23[i,j]
                GG3[i,j+3*M]=-1*Z[i,j]
                GG3[i+M,j+M]=-1*HH32[i,j]+HH42[i,j]
                GG3[i+M,j+2*M]=-1*Y[i,j]
                GG3[i+M,j+3*M]=-1*HH34[i,j]+HH44[i,j]
        
        K1=0
        K0=0

        for j in range(M20):

            if IH[j] == 1:
                K1=K1+1
            if IH[j] == 0:
                if IH[j] == 0:
                    K0=K0+1
                if IH[j] == 1:
                    continue
                   
                for i in range(M10):
                    GG5[i,K0-1]=GG3[i,j]

            else:
                for i in range(M10):
                    GG4[i,K1-1]=-1*GG3[i,j] # Original was -GG3[i,j]
                    B10[K1-1]=B5[j]


        B6 = DMUCRV(GG4,B10)     
        B7 = DLSLCG(GG5,B6)
        
        K1=0

        for j in range(M20):
            if IH[j] == 0:
                K1=K1+1
            if IH[j] == 1:
                continue

            B5[j]=B7[K1-1]

        for i in range(M):
            UK1[i]=B5[i]
            AIK1[i]=B5[i+M]
        
        AA = DMUCRV(Z,AIK1)
        BB = DMUCRV(Y,UK1)
        CC = DMCRCR(LI,LI3)
        DD = DMCRCR(LI,LI2)
        
        for i in range(M):
            B1[i]=UK1[i]
            B1[i+M]=AIK1[i]
            B1[i+2*M]=AA[i]
            B1[i+3*M]=BB[i]
        
        B4 = DLSLCG(GG,B1)
        
        for i in range(M):
            AA[i]=0.
            B[i]=0.
            B[i]=B4[i]

        AA = DMUCRV(LU3,B)

        for i in range(M):
            BB[i]=0.
            B[i]=0.
            B[i]=B4[i+M]

        BB = DMUCRV(LU2,B)

        for i in range(M):
            UX[i]=AA[i]+BB[i]
            if LM == MMT:
                UK1[i]=UX[i]
            UXM[i]=np.sqrt(UX[i].real**2+UX[i].imag**2)

        for i in range(M):
            AA[i]=0.
            B[i]=0.
            B[i]=B4[i+2*M]
        
        AA = DMUCRV(LI3,B)  
        
        for i in range(M):
            BB[i]=0.
            B[i]=0.
            B[i]=B4[i+3*M]
        
        BB = DMUCRV(LI2,B)  
        
        for i in range(M):
                    
            AIX[i]=AA[i]+BB[i]
            if LM == MMT:
                AIK1[i]=AIX[i]
            AIXM[i]=np.sqrt(AIX[i].real**2+AIX[i].imag**2)
            
            if i == 0 and LL == 0 and PR == 2:
                PPP1[NN,LL]=PPP1[NN,LL]+(AIXM[0]**2)/(2*R11[0])
            if i == 0 and LL > 0:
                PPP1[NN,LL]=PPP1[NN,LL]+(AIXM[0]**2)/(2*R11[0])
            if i == 1 and LL == 0 and PR == 2:
                PPP2[NN,LL]=PPP2[NN,LL]+(AIXM[1]**2)/(2*R11[1])
            if i == 1 and LL > 0:
                PPP2[NN,LL]=PPP2[NN,LL]+(AIXM[1]**2)/(2*R11[1])
            if i == 2 and LL == 0 and PR == 2:
                PPP3[NN,LL]=PPP3[NN,LL]+(AIXM[2]**2)/(2*R11[2])
            if i == 2 and LL > 0:
                PPP3[NN,LL]=PPP3[NN,LL]+(AIXM[2]**2)/(2*R11[2])
            if i == 3 and LL == 0 and PR == 2:
                PPP4[NN,LL]=PPP4[NN,LL]+(AIXM[3]**2)/(2*R11[3])
            if i == 3 and LL > 0:
                PPP4[NN,LL]=PPP4[NN,LL]+(AIXM[3]**2)/(2*R11[3])
            if i == 4 and LL == 0 and PR == 2:
                PPP5[NN,LL]=PPP5[NN,LL]+(AIXM[4]**2)/(2*R11[4])
            if i == 4 and LL > 0:
                PPP5[NN,LL]=PPP5[NN,LL]+(AIXM[4]**2)/(2*R11[4])
            if i == 5 and LL == 0 and PR == 2:
                PPP6[NN,LL]=PPP6[NN,LL]+(AIXM[5]**2)/(2*R11[5])
            if i == 5 and LL > 0:
                PPP6[NN,LL]=PPP6[NN,LL]+(AIXM[5]**2)/(2*R11[5])
            if i == 6 and LL == 0 and PR == 2:
                PPP7[NN,LL]=PPP7[NN,LL]+(AIXM[6]**2)/(2*R11[6])
            if i == 6 and LL > 0:
                PPP7[NN,LL]=PPP7[NN,LL]+(AIXM[6]**2)/(2*R11[6])
            if i == 7 and LL == 0 and PR == 2:
                PPP8[NN,LL]=PPP8[NN,LL]+(AIXM[7]**2)/(2*R11[7])
            if i == 7 and LL > 0:
                PPP8[NN,LL]=PPP8[NN,LL]+(AIXM[7]**2)/(2*R11[7])
            if LL == 0 and PR == 2:
                PPP[NN,LL]=PPP[NN,LL]+(AIXM[i]**2)/(2*R11[i])
            if LL > 0:
                PPP[NN,LL]=PPP[NN,LL]+(AIXM[i]**2)/(2*R11[i])
            if LL == 0 and PR == 1:
                PP1=PP1+(AIXM[i]**2)/(2*R11[i])
            if LL == 0 and PR == 2:
                PP2=PP2+(AIX[i]**2)/(2*R11[i])
            SM[i]=UX[i]*np.conj(AIX[i])/2
            
#             print(PPP1[NN, LL])

        # Writing UXM and AIXM values into 10th and 11th modules    

In [11]:
# !!!  470th loop stands for 1th or one prisoed of podstanst  !!!

# 11 - 13
for i in range(560+1):
    for p in range(3):
        UM[i,0,p] = funu[i,p]*np.sqrt(2) 
        FUM[i,0,p]=fu[i,p]*np.pi/180 
        UM1[i,0,p]=UM[i,0,p]*np.cos(FUM[i,0,p]) 
        UM2[i,0,p]=UM[i,0,p]*np.sin(FUM[i,0,p])     
        
# 13 - 14
for i in range(560+1):
    for p in range(3):
        for k in range(50-1):
            UM[i,k+1,p]=UM[i,k+1,p]*UM[i,0,p]/100.
            FUM[i,k+1,p]=FUM[i,k+1,p]*np.pi/180.
            UM1[i,k+1,p]=UM[i,k+1,p]*np.cos(FUM[i,k+1,p])
            UM2[i,k+1,p]=UM[i,k+1,p]*np.sin(FUM[i,k+1,p])

# 14 - 16
for i in range(560+1):
    for p in range(3):
        AIM[i,0,p]=funi[i,p]*np.sqrt(2)
        FIM[i,0,p]=fi[i,p]*np.pi/180
        AIM1[i,0,p]=AIM[i,0,p]*np.cos(FIM[i,0,p])
        AIM2[i,0,p]=AIM[i,0,p]*np.sin(FIM[i,0,p])
        
# 16 - 117
for i in range(560+1):
    for p in range(3):
        for k in range(50-1):
            AIM[i,k+1,p]=AIM[i,k+1,p]*AIM[i,0,p]/100
            FIM[i,k+1,p]=FIM[i,k+1,p]*np.pi/180
            AIM1[i,k+1,p]=AIM[i,k+1,p]*np.cos(FIM[i,k+1,p])
            AIM2[i,k+1,p]=AIM[i,k+1,p]*np.sin(FIM[i,k+1,p])

In [12]:
# 117 - 1700 - 1500

print("!!! Started !!!")

MM, MPR, MTR, DT, MT = 5, 3, 1, 2.5, 5 #         <---- INPUT FROM THE USER
M = MPR + MTR     # thus is 4
MMT = MM / MT    # is 1

if M <= 6:
    M1 = M
if M > 6:
    M1 = 6

M10 = 2 * M # is 8
M20 = 4 * M # is 16

k = LoopI(0, 49) # INNER LOOP
n = LoopI(0, 560) # OUTER LOOP
PR = LoopI(1, 2)
counter = 1

start = time.time()

while True:


    if n.the_end:
        break

    # print("START: ", n.get_current_index(), k.get_current_index())

    K = k.get_current_index(); N = n.get_current_index();
    # print(counter); counter+=1

    UK1[0] = complex(UM1[N,K,0], UM2[N,K,0])
    UK1[1] = complex(UM1[N,K,1], UM2[N,K,1])
    UK1[2] = complex(UM1[N,K,2], UM2[N,K,2])
    UK1[3] = complex(0, 0)
    AIK1[0] = complex(AIM1[N,K,0], AIM2[N,K,0])
    AIK1[1] = complex(AIM1[N,K,1], AIM2[N,K,1])
    AIK1[2] = complex(AIM1[N,K,2], AIM2[N,K,2])
    AIK1[3] = complex(0, 0)


    if not (k.get_current_index() > 0):
        if not (k.get_current_index() == 0 and PR.get_current_index() == 2):
            # print(counter); counter+=1
            UK10=(UK1[0]+UK1[1]+UK1[2])/3
            UK11=(UK1[0]+UK1[1]*AL+UK1[2]*AL**2)/3
            UK12=(UK1[0]+UK1[1]*AL**2+UK1[2]*AL)/3
            SKU2=np.sqrt(UK12.real**2+UK12.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
            SKU0=np.sqrt(UK10.real**2+UK10.imag**2)/np.sqrt(UK11.real**2+UK11.imag**2)*100
            UK1[0]=UK11
            UK1[1]=UK11*AL**2
            UK1[2]=UK11*AL
            AIK10=(AIK1[0]+AIK1[1]+AIK1[2])/3
            AIK11=(AIK1[0]+AIK1[1]*AL+AIK1[2]*AL**2)/3
            AIK12=(AIK1[0]+AIK1[1]*AL**2+AIK1[2]*AL)/3
            SKI2=np.sqrt(AIK12.real**2+AIK12.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
            SKI0=np.sqrt(AIK10.real**2+AIK10.imag**2)/np.sqrt(AIK11.real**2+AIK11.imag**2)*100
            AIK1[1]=AIK11
            AIK1[2]=AIK11*AL**2
            AIK1[3]=AIK11*AL



    # print(n.get_current_index(), k.get_current_index())
    # print(counter); counter+=1
    RASCHET(N, K)


    if k.get_current_index() == 0 and PR.get_current_index() == 1:
        PPR1[N]=PP1
    if k.get_current_index() == 0 and PR.get_current_index() == 2:
        PPR2[N]=PP2
    if k.get_current_index() == 0 and PR.get_current_index() == 1:
        PR.step_up_index()
        continue

    if PR.get_current_index() == 2:
        PR.set_current_index(1)

    if k.the_end:
        n.step_up_index()
        k.set_current_index(0)
    else:
        k.step_up_index()

print("!!! Finished !!!")
end = time.time()
print("Time took(sec): ", end - start)

!!! Started !!!
Y with shape and type:  (4, 4) complex128
0.0
2.210260557582802e-06

0.0
-3.5491508682946704e-07

0.0
-2.990723456477797e-07

0.0
-2.350362530739676e-07

0.0
-3.5491508682946714e-07

0.0
2.2190052794485434e-06

0.0
-3.5033608997717045e-07

0.0
-2.0357154967856418e-07

0.0
-2.9907234564777964e-07

0.0
-3.5033608997717056e-07

0.0
2.26079058186218e-06

0.0
-5.246989355668838e-07

0.0
-2.350362530739677e-07

0.0
-2.0357154967856413e-07

0.0
-5.246989355668839e-07

0.0
2.0498763940696902e-06

Y with shape and type:  (4, 4) complex128
0.0
2.210260557582802e-06

0.0
-3.5491508682946704e-07

0.0
-2.990723456477797e-07

0.0
-2.350362530739676e-07

0.0
-3.5491508682946714e-07

0.0
2.2190052794485434e-06

0.0
-3.5033608997717045e-07

0.0
-2.0357154967856418e-07

0.0
-2.9907234564777964e-07

0.0
-3.5033608997717056e-07

0.0
2.26079058186218e-06

0.0
-5.246989355668838e-07

0.0
-2.350362530739677e-07

0.0
-2.0357154967856413e-07

0.0
-5.246989355668839e-07

0.0
2.0498763940696902e-


0.0
4.30474042754635e-05

Y with shape and type:  (4, 4) complex128
0.0
4.8625732266821645e-05

0.0
-7.808131910248275e-06

0.0
-6.579591604251153e-06

0.0
-5.170797567627287e-06

0.0
-7.808131910248277e-06

0.0
4.8818116147867954e-05

0.0
-7.70739397949775e-06

0.0
-4.478574092928412e-06

0.0
-6.579591604251152e-06

0.0
-7.707393979497753e-06

0.0
4.973739280096796e-05

0.0
-1.1543376582471444e-05

0.0
-5.170797567627289e-06

0.0
-4.478574092928411e-06

0.0
-1.1543376582471445e-05

0.0
4.5097280669533186e-05

Y with shape and type:  (4, 4) complex128
0.0
5.0835992824404446e-05

0.0
-8.163046997077741e-06

0.0
-6.878663949898933e-06

0.0
-5.405833820701255e-06

0.0
-8.163046997077745e-06

0.0
5.10371214273165e-05

0.0
-8.057730069474921e-06

0.0
-4.682145642606976e-06

0.0
-6.878663949898932e-06

0.0
-8.057730069474923e-06

0.0
5.199818338283014e-05

0.0
-1.2068075518038327e-05

0.0
-5.405833820701257e-06

0.0
-4.682145642606975e-06

0.0
-1.2068075518038329e-05

0.0
4.7147157063602874

8.817083269262502e-05

0.0
-2.0463258487108468e-05

0.0
-9.16641386988474e-06

0.0
-7.939290437464e-06

0.0
-2.046325848710847e-05

0.0
7.994517936871793e-05

Y with shape and type:  (4, 4) complex128
0.0
8.841042230331209e-05

0.0
-1.4196603473178682e-05

0.0
-1.1962893825911187e-05

0.0
-9.401450122958704e-06

0.0
-1.4196603473178687e-05

0.0
8.876021117794174e-05

0.0
-1.4013443599086818e-05

0.0
-8.142861987142567e-06

0.0
-1.1962893825911186e-05

0.0
-1.4013443599086822e-05

0.0
9.04316232744872e-05

0.0
-2.098795742267535e-05

0.0
-9.401450122958707e-06

0.0
-8.142861987142566e-06

0.0
-2.0987957422675355e-05

0.0
8.19950557627876e-05

Y with shape and type:  (4, 4) complex128
0.0
9.062068286089489e-05

0.0
-1.455151856000815e-05

0.0
-1.2261966171558967e-05

0.0
-9.636486376032672e-06

0.0
-1.4551518560008153e-05

0.0
9.097921645739027e-05

0.0
-1.4363779689063988e-05

0.0
-8.34643353682113e-06

0.0
-1.2261966171558965e-05

0.0
-1.4363779689063993e-05

0.0
9.269241385634938e-05


Y with shape and type:  (4, 4) complex128
0.0
2.210260557582802e-05

0.0
-3.5491508682946704e-06

0.0
-2.990723456477797e-06

0.0
-2.350362530739676e-06

0.0
-3.5491508682946717e-06

0.0
2.2190052794485434e-05

0.0
-3.5033608997717045e-06

0.0
-2.035715496785642e-06

0.0
-2.9907234564777964e-06

0.0
-3.5033608997717054e-06

0.0
2.26079058186218e-05

0.0
-5.246989355668838e-06

0.0
-2.350362530739677e-06

0.0
-2.0357154967856414e-06

0.0
-5.246989355668839e-06

0.0
2.04987639406969e-05

Y with shape and type:  (4, 4) complex128
0.0
2.4312866133410822e-05

0.0
-3.904065955124138e-06

0.0
-3.2897958021255765e-06

0.0
-2.5853987838136435e-06

0.0
-3.9040659551241385e-06

0.0
2.4409058073933977e-05

0.0
-3.853696989748875e-06

0.0
-2.239287046464206e-06

0.0
-3.289795802125576e-06

0.0
-3.853696989748876e-06

0.0
2.486869640048398e-05

0.0
-5.771688291235722e-06

0.0
-2.5853987838136444e-06

0.0
-2.2392870464642055e-06

0.0
-5.771688291235723e-06

0.0
2.2548640334766593e-05

Y with shape an

-7.051087592219031e-06

0.0
-6.107146490356924e-06

0.0
-1.5740968067006516e-05

0.0
6.149629182209071e-05

Y with shape and type:  (4, 4) complex128
0.0
6.851807728506687e-05

0.0
-1.1002367691713478e-05

0.0
-9.27124271508117e-06

0.0
-7.286123845292996e-06

0.0
-1.1002367691713481e-05

0.0
6.878916366290485e-05

0.0
-1.0860418789292284e-05

0.0
-6.31071804003549e-06

0.0
-9.271242715081169e-06

0.0
-1.0860418789292288e-05

0.0
7.008450803772758e-05

0.0
-1.6265667002573397e-05

0.0
-7.2861238452929984e-06

0.0
-6.310718040035488e-06

0.0
-1.62656670025734e-05

0.0
6.354616821616039e-05

Y with shape and type:  (4, 4) complex128
0.0
7.072833784264967e-05

0.0
-1.1357282778542945e-05

0.0
-9.57031506072895e-06

0.0
-7.5211600983669634e-06

0.0
-1.1357282778542949e-05

0.0
7.100816894235339e-05

0.0
-1.1210754879269455e-05

0.0
-6.514289589714054e-06

0.0
-9.570315060728949e-06

0.0
-1.1210754879269458e-05

0.0
7.234529861958976e-05

0.0
-1.679036593814028e-05

0.0
-7.521160098366966e-

...

Y with shape and type:  (4, 4) complex128
0.0
1.3261563345496812e-05

0.0
-2.1294905209768022e-06

0.0
-1.7944340738866783e-06

0.0
-1.4102175184438058e-06

0.0
-2.1294905209768027e-06

0.0
1.331403167669126e-05

0.0
-2.1020165398630227e-06

0.0
-1.221429298071385e-06

0.0
-1.7944340738866779e-06

0.0
-2.1020165398630236e-06

0.0
1.3564743491173081e-05

0.0
-3.1481936134013024e-06

0.0
-1.4102175184438062e-06

0.0
-1.2214292980713848e-06

0.0
-3.1481936134013033e-06

0.0
1.2299258364418142e-05

Y with shape and type:  (4, 4) complex128
0.0
1.5471823903079616e-05

0.0
-2.4844056078062695e-06

0.0
-2.093506419534458e-06

0.0
-1.6452537715177733e-06

0.0
-2.48440560780627e-06

0.0
1.5533036956139804e-05

0.0
-2.452352629840193e-06

0.0
-1.4250008477499493e-06

0.0
-2.0935064195344575e-06

0.0
-2.452352629840194e-06

0.0
1.582553407303526e-05

0.0
-3.6728925489681863e-06

0.0
-1.6452537715177737e-06

0.0
-1.425000847749949e-06

0.0
-3.672892548968187e-06

0.0
1.4349134758487832e-05

Y with

Y with shape and type:  (4, 4) complex128
0.0
5.7466774497152855e-05

0.0
-9.227792257566143e-06

0.0
-7.775880986842272e-06

0.0
-6.110942579923158e-06

0.0
-9.227792257566146e-06

0.0
5.769413726566213e-05

0.0
-9.108738339406431e-06

0.0
-5.292860291642668e-06

0.0
-7.77588098684227e-06

0.0
-9.108738339406434e-06

0.0
5.878055512841668e-05

0.0
-1.3642172324738977e-05

0.0
-6.11094257992316e-06

0.0
-5.2928602916426674e-06

0.0
-1.364217232473898e-05

0.0
5.3296786245811946e-05

Y with shape and type:  (4, 4) complex128
0.0
5.9677035054735656e-05

0.0
-9.58270734439561e-06

0.0
-8.074953332490052e-06

0.0
-6.345978832997126e-06

0.0
-9.582707344395614e-06

0.0
5.991314254511067e-05

0.0
-9.459074429383603e-06

0.0
-5.496431841321233e-06

0.0
-8.07495333249005e-06

0.0
-9.459074429383604e-06

0.0
6.104134571027886e-05

0.0
-1.4166871260305861e-05

0.0
-6.345978832997127e-06

0.0
-5.4964318413212315e-06

0.0
-1.4166871260305865e-05

0.0
5.5346662639881634e-05

Y with shape and type: 

0.0
-1.0576631388328543e-05

0.0
-1.597117890732602e-05

0.0
9.985523757518445e-05

0.0
-1.576512404897267e-05

0.0
-9.160719735535389e-06

0.0
-1.3458255554150084e-05

0.0
-1.5765124048972675e-05

0.0
0.0001017355761837981

0.0
-2.361145210050977e-05

0.0
-1.0576631388328546e-05

0.0
-9.160719735535385e-06

0.0
-2.3611452100509774e-05

0.0
9.224443773313607e-05

Y with shape and type:  (4, 4) complex128
0.0
0.00010167198564880889

0.0
-1.6326093994155483e-05

0.0
-1.3757327899797865e-05

0.0
-1.081166764140251e-05

0.0
-1.632609399415549e-05

0.0
0.000102074242854633

0.0
-1.6115460138949842e-05

0.0
-9.364291285213952e-06

0.0
-1.3757327899797864e-05

0.0
-1.6115460138949845e-05

0.0
0.00010399636676566028

0.0
-2.4136151036076655e-05

0.0
-1.0811667641402514e-05

0.0
-9.36429128521395e-06

0.0
-2.4136151036076658e-05

0.0
9.429431412720575e-05

Y with shape and type:  (4, 4) complex128
0.0
0.00010388224620639169

0.0
-1.6681009080984952e-05

0.0
-1.4056400245445645e-05

0.0
-1.10467

0.0
2.2548640334766593e-05

Y with shape and type:  (4, 4) complex128
0.0
2.6523126690993623e-05

0.0
-4.2589810419536045e-06

0.0
-3.5888681477733566e-06

0.0
-2.8204350368876115e-06

0.0
-4.258981041953605e-06

0.0
2.662806335338252e-05

0.0
-4.2040330797260454e-06

0.0
-2.44285859614277e-06

0.0
-3.5888681477733557e-06

0.0
-4.204033079726047e-06

0.0
2.7129486982346162e-05

0.0
-6.296387226802605e-06

0.0
-2.8204350368876124e-06

0.0
-2.4428585961427696e-06

0.0
-6.2963872268026065e-06

0.0
2.4598516728836285e-05

Y with shape and type:  (4, 4) complex128
0.0
2.8733387248576427e-05

0.0
-4.613896128783071e-06

0.0
-3.887940493421136e-06

0.0
-3.055471289961579e-06

0.0
-4.613896128783073e-06

0.0
2.8847068632831064e-05

0.0
-4.5543691697032155e-06

0.0
-2.646430145821334e-06

0.0
-3.887940493421135e-06

0.0
-4.554369169703217e-06

0.0
2.939027756420834e-05

0.0
-6.821086162369489e-06

0.0
-3.05547128996158e-06

0.0
-2.6464301458213337e-06

0.0
-6.82108616236949e-06

0.0
2.664839312

KeyboardInterrupt: 

In [ ]:
# 1500 - 1501

for n in range(560):
    PRP = 0
    for k in range(50): # 1060'th loop
        PRP=PRP+PPP[n,k]
    RPR=0
    for k in range(1, 50): # 1061'st loop
        RPR=RPR+PPP[n,k]
    
    SS1=(PPP[n,0]/PPR1[n]-1)*100
    SS2=(RPR/PPR1[n])*100
    SS0=PPP[n,0]-PPR1[n]
    
    # WRITING INTO "PPP" and "PPPx" files corresponding PPP and PPPx values
#     print(PPP1[n, k])
    
# 1501 - 1052
WD0=0.
for k in range(50):
    WD[0,k]=0
    for i in range(560):
        WD0=WD0+PPP[i,k]*DT/60000
        WD[0,k]=WD[0,k]+PPP[i,k]*DT/60000

# 1052 - 1057
WD1=WD0-WD[0,0]
for k in range(50):
    WD[1,k]=WD[0,k]/WD0*100
WD4=0
for k in range(13, 50):
    WD4=WD4+WD[1,k]
WD10=0
for i in range(560):
    WD10=WD10+PPR1[i]*DT/60000
 
# 1057 -1054
for i in range(560):
    PD[0,i] = 0
    PD[1,i] = 0
    for k in range(50):
        PD[0,i]=PD[0,i]+PPP[i,k]
    PD[1,i]=PD[0,i]-PPP[i,0]
    PD[2,i]=PD[1,i]/PD[0,i]*100

# WRITING CALCULATION RESULTS INTO 8th UNIT (Результаты расчета)

In [ ]:
for item in PPP1[:560,1]:
    print(item)

In [ ]:
int(WD0) # HAVE TO BE AROUND ~ 16.79